# The Hangman Game


In [ ]:
import pandas as pd
import numpy as np
import os
import random
import string
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from itertools import combinations
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from utilities import *
from model import *

In [ ]:
# prepare the data for LSTM
input_tensor, target_tensor = process_and_prepare_datasets()

2-gram is Done!
3-gram is Done!
4-gram is Done!
5-gram is Done!
6-gram is Done!
Input data saved successfully.
Target data saved successfully.
torch.Size([29977280, 6]) torch.Size([29977280, 26])


In [ ]:
# train the model
train_model(input_tensor, target_tensor)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1
-------------------------------
loss: 16.674883  [  128/29977280]
loss: 16.267006  [128128/29977280]
loss: 15.784765  [256128/29977280]
loss: 15.513313  [384128/29977280]
loss: 15.323220  [512128/29977280]
loss: 15.203616  [640128/29977280]
loss: 15.313610  [768128/29977280]
loss: 15.335670  [896128/29977280]
loss: 15.396443  [1024128/29977280]
loss: 15.493000  [1152128/29977280]
loss: 15.203526  [1280128/29977280]
loss: 15.059254  [1408128/29977280]
loss: 14.934552  [1536128/29977280]
loss: 14.840385  [1664128/29977280]
loss: 15.151722  [1792128/29977280]
loss: 14.850445  [1920128/29977280]
loss: 15.199343  [2048128/29977280]
loss: 15.356495  [2176128/29977280]
loss: 15.289555  [2304128/29977280]
loss: 15.281339  [2432128/29977280]
loss: 15.127481  [2560128/29977280]
loss: 14.914280  [2688128/29977280]
loss: 15.089948  [2816128/29977280]
loss: 15.076271  [2944128/29977280]
loss: 14.877400  [3072128/29977280]
loss: 15.212141  [3200128/29977280]
loss: 14.767307  [3328128/2997728

In [ ]:
def encode_input_for_prediction(masked_word):
    """
    Encode the masked word.

    Parameters:
    - masked_word (str): The word with masked characters.

    Returns:
    - torch.Tensor: Encoded input tensor for prediction.
    """
    # Create a character-to-index mapping and an underscore placeholder
    char_to_index, _ = create_char_mapping()

    # Encode the masked word using the char_to_index mapping
    input_data = [encode_input(masked_word)]
    input_tensor = torch.tensor(input_data, dtype=torch.long)
    return input_tensor


def extract_ngrams(word):
    """
    Make all possible n-grams from the word with at least one underscore.

    Parameters:
    - word (str): The input word.

    Returns:
    - list: List of unique n-grams.
    """
    ngrams = set()

    # Iterate over different n-gram lengths (2, 3, 4, 5, 6)
    for n in range(2, 7):
        # Extract n-grams from the word
        for i in range(len(word) - n + 1):
            ngram = word[i:i + n]

            # Check if the n-gram contains at least one alphabet and one underscore
            if any(char.isalpha() for char in ngram) and '_' in ngram:
                # If the n-gram is shorter than 6, pad it with asterisks
                ngram = ngram.ljust(6, '*')

                # Ensure the n-gram is of length 6
                ngram = ngram[:6]

                # Add the n-gram to the set
                ngrams.add(ngram)

    return list(ngrams)

def encode_ngram(ngram, char_to_index):
    """
    Encode a given n-gram using a character-to-index mapping.

    Parameters:
    - ngram (str): The input n-gram.
    - char_to_index (dict): Character-to-index mapping.

    Returns:
    - list: Encoded n-gram.
    """
    # Ensure the n-gram is of length 6
    ngram = ngram[:6]

    # Encode each character in the n-gram using char_to_index mapping
    encoded_ngram = [char_to_index[char] for char in ngram]

    return encoded_ngram

def get_sorted_letters(new_dictionary, guessed_letters):
    """
    Get sorted letters based on their frequency in the new dictionary.

    Parameters:
    - new_dictionary (list): List of possible words.
    - guessed_letters (list): List of guessed letters.

    Returns:
    - list: Sorted letters based on frequency, excluding guessed letters.
    """
    full_dict_string = "".join(new_dictionary)

    # Count the occurrences of each letter
    c = collections.Counter(full_dict_string)

    sorted_letter_count = c.most_common()

    # Filter out guessed letters
    remaining_sorted_letters = [item for item in sorted_letter_count if item[0] not in guessed_letters]

    return remaining_sorted_letters

def func(new_dictionary):
    """
    Count the occurrences of each letter in the new dictionary.

    Parameters:
    - new_dictionary (list): List of possible words.

    Returns:
    - collections.Counter: Count of occurrences for each letter.
    """
    dictx = collections.Counter()
    for words in new_dictionary:
        temp = collections.Counter(words)
        for i in temp:
            temp[i] = 1
            dictx = dictx + temp
    return dictx

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        full_dictionary_location = "/content/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.tries_remains  = 6
        self.model = NeuralNetwork()
        self.model.load_state_dict(torch.load("/content/lstm_ngram_2.pt"))
        self.ngram_used = set()
        self.LSTM_guess = 0


    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link


    def predicted_letter_lstm(self, masked_word):
        """
        Predict the next letter using the LSTM model based on the masked word.

        Parameters:
        - masked_word (str): The word with masked characters.

        Returns:
        - list: Predicted letters sorted by probability.
        """
        # Create character mappings
        char_to_index, int_to_char = create_char_mapping()

        # Extract unique n-grams with at least one alphabet
        ngrams = extract_ngrams(masked_word)

        # Initialize an empty dictionary to store accumulated probabilities
        accumulated_probabilities = {}

        # Traverse over n-grams
        for ngram in ngrams:
            # Encode the n-gram
            input_tensor_for_prediction = encode_ngram(ngram, char_to_index)

            # Convert to tensor
            input_tensor = torch.tensor(input_tensor_for_prediction, dtype=torch.long)
            input_tensor = input_tensor.view(1, -1)

            # Ensure the model is in evaluation mode
            self.model.eval()

            # Make predictions
            with torch.no_grad():
                output = self.model(input_tensor)

            # Apply softmax to get probabilities
            probabilities = torch.softmax(output, dim=1).numpy()

            # Process the probabilities using int_to_char
            probabilities_list = [(int_to_char[i], prob) for i, prob in enumerate(probabilities[0])]

            alphabet_count = sum(1 for char in ngram if char.isalpha())

            # Give more weight to the new built n-gram (new information)
            if ngram not in self.ngram_used:
                self.ngram_used.add(ngram)
                weight = 2
            else:
                weight = 1

            # Accumulate probabilities for each alphabet
            for char, prob in probabilities_list:
                accumulated_probabilities[char] = accumulated_probabilities.get(char, 0) + prob * weight

        # Convert the accumulated probabilities to a list of tuples
        final_accumulated_list = list(accumulated_probabilities.items())

        sorted_probabilities_list = sorted(final_accumulated_list, key=lambda x: x[1], reverse=True)

        sorted_letters_list = [pred for pred, _ in sorted_probabilities_list]

        return sorted_letters_list


    def guess(self, word):
        # word input example: "_ p p _ e "

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")

        # find length of passed word
        len_word = len(clean_word)

        # remaing spaces
        remaining_spaces = clean_word.count('.')

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.full_dictionary
        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        current_dictionary = new_dictionary

        # start the guess letter
        guess_letter = '!'

        # if we have not yet guessed at least 2, start with most common letters,
        # in the dictionary of the words with the same length
        if (len_word - remaining_spaces) < 2:
            full_dict_string = "".join(new_dictionary)
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            c = collections.Counter(full_dict_string)
            sorted_letter_count = c.most_common()
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        remaining_sorted_letters = get_sorted_letters(new_dictionary, self.guessed_letters)

        # now we have at least two letters, use LSTM:
        if guess_letter == '!':
            predict_letters = self.predicted_letter_lstm(word[::2])
            for letter in predict_letters:
                # check if the prediction is alphabet and it is not already suggested
                if letter.isalpha() and letter not in self.guessed_letters:
                    guess_letter = letter
                    break


        # if there was no match: based on words with the same pattern:
        if guess_letter == '!' or guess_letter == '_':
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            c = func(new_dictionary)
            sorted_letter_count = c.most_common()
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!' or guess_letter == '_':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,_ in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            self.tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, self.tries_remains, word))
            while self.tries_remains > 0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                self.tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)